# Animation of the COVID-19 spread

We use the data as provided by https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases which was retrieved on March 19th 2020.

A mapping is created using common python geography libraries. This mapping is done via a country code. However, some entries may be missing. This animation is provided as is and should not be used as a source of information.

In [32]:
import pandas as pd

data_path = './input/time_series__ncov_confirmed.csv'
data = pd.read_csv(data_path)

# Clean the data
data.index = data['Country/Region']
data = data.drop(columns=['Country/Region', 'Province/State', 'Lat', 'Long'])
data = data.groupby('Country/Region').sum()
data.columns = pd.to_datetime(data.columns)

Create a mapping from the country name in the dataset to the ISO alpha 3 standard.

In [33]:
import pycountry

country_codes_a2 = []
country_codes_a3 = []

for country in data.index:
    if 'Congo' in country:
        country = 'Congo'    
    try:
        cdata = pycountry.countries.search_fuzzy(country)[0]
        country_code_a3 = cdata.alpha_3
        country_code_a2 = cdata.alpha_2
    except:
        country_code_a2 = 'XX'
        country_code_a3 = 'XXX'
     
    country_codes_a2.append(country_code_a2)
    country_codes_a3.append(country_code_a3)

Add this to the dataframe

In [34]:
data['country_code'] = country_codes_a3
data = data.groupby('country_code').sum()

In [38]:
import numpy as np

# For all geography plotting
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Set a nice font size
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 22}

mpl.rc('font', **font)

# Load the counties
shapename = 'admin_0_countries'
countries_shp = shpreader.natural_earth(resolution='110m',
                                        category='cultural', name=shapename)

# Set the colormap
cmap = cm.Reds
colors_max = 6 # We use a log 10 scaling, so 6 should be seen as 10^6 i.e. 100 k confirmed cases

for i, col in enumerate(data.columns): 
    
    # Get the day and the number of cases from the data
    today = col
    countries_cases_today = dict(zip(data[col].index, data[col].values))
    
    # Create a base map with required attributes
    fig = plt.figure(figsize=(16,9))
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_title(today.date())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.COASTLINE, linewidth=2)
    ax.add_feature(cartopy.feature.BORDERS, linewidth=2, linestyle='-')
    
    
    for country in shpreader.Reader(countries_shp).records():
        # Get the country name and ISO_A3 code
        # The latter is used to map to our known dataset
        country_name = country.attributes['NAME']
        country_id = country.attributes['ADM0_A3']
        
        try:
            country_cases = countries_cases_today[country_id]
        except:
            country_cases = 0
        
        # Use a logarithmic mapping
        color_val = np.log10(country_cases + 1)/colors_max
        ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                          facecolor=cmap(color_val),
                          label=country_name)

    # Make sure to use the same logarithmic mapping here
    norm = mpl.colors.SymLogNorm(1, vmin=1, vmax=10**colors_max)
    sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
    
    # Add a colorbar
    cbar = plt.colorbar(sm, ax=ax, label='confirmed COVID-19 case #')
    
    # Add reference and name
    plt.text(-150, -120, 'Source: https://data.humdata.org/dataset/\nnovel-coronavirus-2019-ncov-cases')
    plt.text(-150, -50, 'VH', color='white', size=50)
    
    # Save as individual pngs
    plt.savefig(f'./output/{i:03}', dpi=150)
    plt.close()

Combine pngs into a gif and mp4 movie

In [39]:
import imageio
from glob import glob

global_duration = 0.1
with imageio.get_writer('movie.gif', mode='I', duration=global_duration) as writer:
    filenames = glob('./output/*')
    durations = np.logspace(0, 3, len(filenames), base=10)
    durations = durations/np.max(durations)/2

    for duration, filename in zip(durations, filenames):
        image = imageio.imread(filename)
        
        for j in range(int(np.ceil(duration/global_duration))):
            writer.append_data(image)
            
            
with imageio.get_writer('movie.mp4', mode='I', fps=1/global_duration) as writer:
    filenames = glob('./output/*')
    durations = np.logspace(0, 3, len(filenames), base=10)
    durations = durations/np.max(durations)/2

    for duration, filename in zip(durations, filenames):
        image = imageio.imread(filename)
        
        for j in range(int(np.ceil(duration/global_duration))):
            writer.append_data(image)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2400, 1350) to (2400, 1360) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Optimize the gif size